FastText Pre-trained 한국어 모델 사용기
- https://inahjeon.github.io/fasttext/

In [4]:
from gensim import models

path = 'resource/cc.ko.300.bin'
ko_model = models.fasttext.load_facebook_model(path)




In [5]:
for w, sim in ko_model.similar_by_word('파이썬', 10):
    print(f'{w}: {sim}')


/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:1: DeprecationWarning: Call to deprecated `similar_by_word` (Method will be removed in 4.0.0, use self.wv.similar_by_word() instead).
  if __name__ == '__main__':


Python: 0.565061628818512
자이썬: 0.5624369382858276
레일스: 0.5598082542419434
파이썬을: 0.5595801472663879
언어용: 0.5288202166557312
파이썬의: 0.5250024795532227
프로그래밍: 0.5225088000297546
wxPython: 0.5222088098526001
파이썬이나: 0.5201171636581421
함수형: 0.5187377333641052


In [9]:
vector = ko_model.wv.get_vector('과일')
vector.shape

(300,)

In [17]:
##################################
# FastText Word2Vec 관련 라이브러리
##################################
from scipy import spatial
from konlpy.tag import Okt
Okt = Okt()
pretrained_kr_word2vec = 'resource/ko.bin'
# datapath = 'ko.bin'
datapath = pretrained_kr_word2vec
# print(datapath)
 
import gensim
import os
import numpy as np

word2vec = gensim.models.Word2Vec.load(datapath)

# tokenizer : 문장에서 색인어 추출을 위해 명사,동사,형용사, 부사, 알파벳 정도의 단어만 뽑아서 normalization, stemming 처리하도록 함
def tokenizer(raw, pos=["Noun","Alpha","Verb","Adjective", "Adverb"], stopword=[]):
    return [
        word for word, tag in Okt.pos(
            raw, 
            norm=True,   # normalize 그랰ㅋㅋ -> 그래ㅋㅋ
            stem=True    # stemming 바뀌나->바뀌다
            )
            if len(word) > 1 and tag in pos and word not in stopword
        ]


from gensim import models
def get_fasttext_model(path):
    if type(ko_model) == gensim.models.fasttext.FastText:
        print('Model is already loaded')
    else:
        path = 'resource/cc.ko.300.bin'
    ko_model = get_fasttext_model(path)            
    ko_model = models.fasttext.load_facebook_model(path)
    return ko_model


def sim_sentences_fasttext_word2vec(ko_model, sentences):
    '''
    word2vect의 리터하는 벡터들의 평균을 구하고, 이를 코사인 거리를 구함
    '''
    def get_word2vec(ko_model, words, embedding_size=300):
        '''
        단어 리스트를 받고, 벡터로 변환후에 모든 벡터의 평균을 리턴
        '''
        vectors = []
        for i, word in enumerate(words):
            try:
                vector = ko_model.wv.get_vector(word)
                # print(word)
            except:
                # print("Unknowd words")
                vector = np.zeros(embedding_size) # 모르는 단어이면 0으로 채움
            # break
            vectors.append(vector)
            # print(vector.sum())     

        avg_vectors = np.mean(vectors, axis=0)    
        return avg_vectors
    
    s1 = sentences[0]
    s2 = sentences[1]
    ps1 = tokenizer(s1)
    ps2 = tokenizer(s2)

    avg_vector1 = get_word2vec(ko_model, ps1)
    avg_vector2 = get_word2vec(ko_model, ps2)
    # print(avg_vector)
    dist_w2v = spatial.distance.cosine(avg_vector1, avg_vector2)
    print(s1,',', s2)
    print('dist_w2v: {}'.format(dist_w2v))    

    

In [18]:
s1 ='사과는 과일이다'
s2 = '책은 인류가 쌓은 지식의 보고다'
s3 = '건강에 좋은 것은 과일이다'

sents1 = [s1, s2]    
sents2 = [s1, s3]    

sim_sentences_fasttext_word2vec(ko_model, sents1)
sim_sentences_fasttext_word2vec(ko_model, sents2)


사과는 과일이다 , 책은 인류가 쌓은 지식의 보고다
dist_w2v: 0.680292159318924
사과는 과일이다 , 건강에 좋은 것은 과일이다
dist_w2v: 0.3414410352706909


In [30]:
from gensim import models
def get_fasttext_model(ko_model, path):
    '''
    path = 'resource/cc.ko.300.bin'
    ko_model = get_fasttext_model(path)    
    '''    
    try:
        if type(ko_model) == gensim.models.fasttext.FastText:
            print('Model is already loaded')
    except:
        print('Model is loading')
        ko_model = models.fasttext.load_facebook_model(path)
        
    return ko_model        

path = 'resource/cc.ko.300.bin'
ko_model = get_fasttext_model(ko_model, path)    


Model is already loaded


In [35]:
try:
    ko_model == None
except:
    ko_model = None
    path = 'resource/cc.ko.300.bin'    
    ko_model = get_fasttext_model(ko_model, path)    

sim_sentences_fasttext_word2vec(ko_model, sents1)
sim_sentences_fasttext_word2vec(ko_model, sents2)



사과는 과일이다 , 책은 인류가 쌓은 지식의 보고다
dist_w2v: 0.680292159318924
사과는 과일이다 , 건강에 좋은 것은 과일이다
dist_w2v: 0.3414410352706909
